In [45]:
import pandas as pd
import os

In [ ]:
DATA_PATH = os.path.join("..", "..", "data", "ocupacion_hotelera")

In [46]:
df = pd.read_csv("2942.csv", sep=";")

In [47]:
df.head()

,Tipo de alojamiento,Total Nacional,Comunidades y Ciudades Autonomas,Establecimientos y personal empleado (plazas),Periodo,Total
0,Encuesta de Ocupacion Hotelera,Total Nacional,NaN,Numero de establecimientos abiertos estimados,2025M01,11.362
1,Encuesta de Ocupacion Hotelera,Total Nacional,NaN,Numero de establecimientos abiertos estimados,2024M12,12.155
2,Encuesta de Ocupacion Hotelera,Total Nacional,NaN,Numero de establecimientos abiertos estimados,2024M11,13.031
3,Encuesta de Ocupacion Hotelera,Total Nacional,NaN,Numero de establecimientos abiertos estimados,2024M10,15.351
4,Encuesta de Ocupacion Hotelera,Total Nacional,NaN,Numero de establecimientos abiertos estimados,2024M09,16.529


In [48]:
df.columns = [
    "tipo_alojamiento",
    "total_nacional",
    "comunidad_autonoma",
    "categoria",
    "periodo",
    "valor",
]

df.head()

,tipo_alojamiento,total_nacional,comunidad_autonoma,categoria,periodo,valor
0,Encuesta de Ocupacion Hotelera,Total Nacional,NaN,Numero de establecimientos abiertos estimados,2025M01,11.362
1,Encuesta de Ocupacion Hotelera,Total Nacional,NaN,Numero de establecimientos abiertos estimados,2024M12,12.155
2,Encuesta de Ocupacion Hotelera,Total Nacional,NaN,Numero de establecimientos abiertos estimados,2024M11,13.031
3,Encuesta de Ocupacion Hotelera,Total Nacional,NaN,Numero de establecimientos abiertos estimados,2024M10,15.351
4,Encuesta de Ocupacion Hotelera,Total Nacional,NaN,Numero de establecimientos abiertos estimados,2024M09,16.529


In [49]:
# Drop rows with NaN values
df = df.drop(["total_nacional"], axis=1)
df["comunidad_autonoma"] = df["comunidad_autonoma"].astype(str).str.strip()
df = df.drop(df[df.comunidad_autonoma == "nan"].index)

In [50]:
# Remove leading numbers from comunidad_autonoma
df["comunidad_autonoma"] = (
    df["comunidad_autonoma"].astype(str).str.replace(r"^\d+\s+", "", regex=True)
)

In [51]:
# Parse datetime
df["periodo"] = pd.to_datetime(df["periodo"], format="%YM%m").dt.strftime("%Y-%m-%d")


In [52]:
# Remove dots from numbers
df["valor"] = df["valor"].astype(str).str.replace(r"\.", "", regex=True)

In [53]:
df["tipo_alojamiento"].unique()

old_row_values = [
    "Encuesta de Ocupacion Hotelera",
    "Encuesta de Ocupacion en Campings",
    "Encuesta de Ocupacion en Apartamentos Turisticos",
    "Encuesta de Ocupacion en Alojamientos de Turismo Rural",
    "Encuesta de Ocupacion en Albergues",
]

new_row_values = [
    "hotel",
    "camping",
    "apartamento_turistico",
    "turismo_rural",
    "albergue",
]

for old, new in zip(old_row_values, new_row_values):
    df = df.replace(old, new)

df.head()

,tipo_alojamiento,comunidad_autonoma,categoria,periodo,valor
867,hotel,Andalucia,Numero de establecimientos abiertos estimados,2025-01-01,1983
868,hotel,Andalucia,Numero de establecimientos abiertos estimados,2024-12-01,2143
869,hotel,Andalucia,Numero de establecimientos abiertos estimados,2024-11-01,2277
870,hotel,Andalucia,Numero de establecimientos abiertos estimados,2024-10-01,2599
871,hotel,Andalucia,Numero de establecimientos abiertos estimados,2024-09-01,2785


In [54]:
df["categoria"].unique()

old_row_values = [
    "Numero de establecimientos abiertos estimados",
    "Numero de plazas estimadas",
    "Personal empleado",
]

new_row_values = ["establecimientos_abiertos", "plazas", "personal_empleado"]

for old, new in zip(old_row_values, new_row_values):
    df = df.replace(old, new)

In [55]:
# Replace 'nan' with NaN
df = df.replace("nan", pd.NA)

In [56]:
df["valor"] = pd.to_numeric(df["valor"], errors="coerce")

df_pivoted = df.pivot_table(
    index=["tipo_alojamiento", "comunidad_autonoma", "periodo"],
    columns="categoria",
    values="valor",
).reset_index()


In [57]:
df_pivoted

categoria,tipo_alojamiento,comunidad_autonoma,periodo,establecimientos_abiertos,personal_empleado,plazas
0,albergue,Andalucia,2014-01-01,11.0,168.0,2410.0
1,albergue,Andalucia,2014-02-01,10.0,166.0,2446.0
2,albergue,Andalucia,2014-03-01,13.0,200.0,2878.0
3,albergue,Andalucia,2014-04-01,17.0,218.0,3703.0
4,albergue,Andalucia,2014-05-01,18.0,219.0,3840.0
...,...,...,...,...,...,...
21816,turismo_rural,"Rioja, La",2024-09-01,100.0,139.0,1036.0
21817,turismo_rural,"Rioja, La",2024-10-01,99.0,141.0,1056.0
21818,turismo_rural,"Rioja, La",2024-11-01,95.0,138.0,1028.0
21819,turismo_rural,"Rioja, La",2024-12-01,99.0,134.0,1077.0


In [58]:
# Write to CSV
df_pivoted.to_csv(os.path.join(DATA_PATH, "2942_clean.csv"), index=False)